# Worksheet 05

Name:  
UID: 

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

- 1st Iter: {0, .5}, {1.5, 2, 6, 6.5, 7}
- New Centroids: [.25, 4.6]
- 2nd Iter: {0, .5, 1.5, 2}, {6, 6.5, 7}
- New Centroids: [1, 6.5]
- 3rd Iter: {0, .5, 1.5, 2}, {6, 6.5, 7}
- New Centroids: [1. 6.5]
- Centroids unchanged, algorithm complete.

b) Describe in plain english what the cost function for k means is.

The cost function for k-means is a function that evaluates how well/accurate the clusters are in grouping the data.

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

Different places for the initial centroids could lead to the clusters converging in different places.

d) Does Lloyd's Algorithm always converge? Why / why not?

No - in some cases, it can get stuck in an infinite loop where an arbitrary point A is closer to centroid X, then after new centroids are averaged out, it is suddenly closer to centroid Y, then back to X in the next iteration, etc.

e) Follow along in class the implementation of Kmeans

In [4]:
# Credit to Anonymous Beaker from @148 in Piazza, I had an internship interview on the afternoon of this lecture.
# This is not my code, and is here to help me understand the implementation of K-Means.

import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))
        
    def is_unassigned(self, i):
        return self.assignment[i] == -1
    
    def unassign_all(self):
        self.assignment = [-1 for _ in range(len(self.data))]
        
    def initialize(self):
        return self.data[np.random.choice(range(len(self.data)), size=self.k, replace=False)]
    
    def are_centers_diff(self, c1, c2):
        for i in range(len(c1)):
            if c1[i] not in c2:
                return True
        return False
    
    def assign(self, centers):
        for i in range(len(self.data)):
            self.assignment[i] = 0
            temp_assign = 0
            temp_dist = self.dist(self.data[i], centers[0])
            for j in range(1, len(centers)):
                new_dist = self.dist(self.data[i], centers[j])
                if temp_dist > new_dist:
                    self.assignment[i] = j
                    temp_dist = new_dist
    
    def calculate_new_centers(self):
        centers = []
        for j in range(self.k):
            cluster_j = self.data[
                np.array([i for i in range(len(self.data)) if self.assignment[i] == j])
            ]
            centers.append(np.mean(cluster_j,axis=0))
        
        return np.array(centers)

    def dist(self, x, y):
        return sum((x - y) ** 2) ** (1/2)

    def lloyds(self):
        centers = self.initialize()
        self.assign(centers)
        self.snap(centers)
        new_centers = self.calculate_new_centers()
        while self.are_centers_diff(centers, new_centers):
            centers = new_centers
            self.snap(centers)
            self.unassign_all()
            self.assign(centers)
            new_centers = self.calculate_new_centers()
            print (new_centers)
        return
            
kmeans = KMeans(X, 4)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)

[[ 1.76383784 -0.67432696]
 [-0.28651321 -2.54977694]
 [-0.72799553  1.71532369]
 [ 2.08276048 -4.4674741 ]]
[[ 1.93879912  0.60933927]
 [-0.22086978 -2.28586857]
 [-1.31001536  1.7026119 ]
 [ 2.05028912 -4.33232183]]
[[ 1.76121587  1.41390616]
 [-0.0952648  -1.60885295]
 [-2.27755769  1.65312148]
 [ 2.02246489 -4.24032477]]
[[ 1.6374103   1.64810707]
 [-0.03303221 -1.0148699 ]
 [-2.76197542  1.70053332]
 [ 1.97985835 -4.18780266]]
[[ 1.71222179  1.87128886]
 [ 0.1020831  -0.44327502]
 [-2.90101537  1.7680821 ]
 [ 1.87039274 -4.15046079]]
[[ 1.79136962  1.93703117]
 [ 0.02653564 -0.19437425]
 [-3.00866911  1.82497252]
 [ 1.84012274 -4.13675116]]
[[ 1.88073218  1.9820243 ]
 [ 0.00773286 -0.06380801]
 [-3.04862622  1.85340542]
 [ 1.84012274 -4.13675116]]
[[ 1.91248297e+00  2.00204407e+00]
 [ 1.33733963e-02 -3.71451966e-04]
 [-3.07703402e+00  1.85266958e+00]
 [ 1.84012274e+00 -4.13675116e+00]]
[[ 1.91248297e+00  2.00204407e+00]
 [ 1.33733963e-02 -3.71451966e-04]
 [-3.07703402e+00  1.85266